In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import bq_helper
from plotly import graph_objects as go

from google.cloud import bigquery
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In the preceding cell, I have imported all the essential libraries for data manipulation and visualizing analytics. Specifically, I've harnessed the power of 'Pandas' and 'Numpy' for efficient data manipulation, while 'Matplotlib,' 'Seaborn,' and 'Plotly' have proven to be valuable tools for visualizing our content. Additionally, I've leveraged the Google Cloud platform, specifically the BigQuery service, to process our data, and I've established a connection using the 'bq_helper' and 'client API' for this purpose.

#  Conversion Funnel Analysis

Analyzing the comprehensive metrics of customer interactions on our website is of paramount importance. Google Analytics proves to be invaluable in capturing these insights. By closely examining customer actions, we can readily identify pain points, areas for improvement on the website, and gain fresh ideas to attract and engage our customers.

**Note:** *It's important to acknowledge the limitations of Google Analytics, as it may not capture customer data when ad blockers are in use or if users haven't accepted cookies. Therefore, while using this data for insights, it's essential to keep in mind that the information provided may not always be entirely accurate, but it still serves as a valuable tool for gaining an overview of website activity.*

In [3]:
# writing query to get Visitors & Visits by action taken

action_Query="""
SELECT 
hits.eCommerceAction.action_type AS actions,
COUNT(DISTINCT fullVisitorId) AS n_visitors,
COUNT(DISTINCT visitId) AS n_visits
FROM 
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) AS hits
WHERE 
hits.eCommerceAction.action_type ='1'
OR hits.eCommerceAction.action_type ='2'
OR hits.eCommerceAction.action_type ='3'
OR hits.eCommerceAction.action_type ='5'
OR hits.eCommerceAction.action_type ='6'

GROUP BY actions
ORDER BY n_visits DESC
"""
# This query results a series of data, we need to convert this data into dataframe. AS datafram give advantages to work with different tools in upcoming
visits_df = client.query(action_Query).result().to_dataframe()

#Visualizing the few sessions
visits_df.head()

,actions,n_visitors,n_visits
0,1,99502,123993
1,2,99256,123692
2,3,39817,50022
3,5,18280,22371
4,6,10022,11549


From the above query, the result is typically presented as a dataframe or table. Google Analytics records data using predefined numeric codes. For a clearer understanding of the schema and the meaning behind these codes, you can refer to this [Link](https://support.google.com/analytics/answer/3437719?hl=en). To make the data more interpretable, it is necessary to convert these numeric codes into distinct action names as outlined below.

In [4]:
    #Map the action names 
    action_dic={'1':'Click product lists',
               '2':'View product details', 
               '3':'Add to cart',
               '4':'Remove from cart',
               '5':'Check out', 
               '6':'Purchase',
               '7':'Refund',
               '8':'Unknown'}
    visits_df['actions']=visits_df['actions'].map(action_dic)
    visits_df.head()

,actions,n_visitors,n_visits
0,Click product lists,99502,123993
1,View product details,99256,123692
2,Add to cart,39817,50022
3,Check out,18280,22371
4,Purchase,10022,11549


Now that we have acquired the data, it's time to visualize it. For this purpose, I've opted to use [Plotly Funnel](https://plotly.com/python/funnel-charts/) visuals. Funnel visuals excel in illustrating the flow of data and conversion rates as we move from one stage to the next in a way that provides valuable insights into user interactions.

In [6]:
#initialize the plotly figure
fig=go.Figure()
#Adding the funnel chart traces 
fig.add_trace(go.Funnel(
                        y=visits_df['actions'],
                        x=visits_df['n_visitors'],
                        textinfo= "value+percent previous"))
#Update the layout of the chart
fig.update_layout(title_text = 'Visitors Funnel - % of previous step')
fig.show()


Visit funnel

In [6]:
fig=go.Figure()
#Adding the funnel chart traces 
fig.add_trace(go.Funnel(
                        y=visits_df['actions'],
                        x=visits_df['n_visits'],
                        textinfo= "value+percent previous"))
#Update the layout of the chart
fig.update_layout(title_text = 'Visits Funnel - % of previous step')
fig.show()

Two funnel charts above depict the total number of unique visitors and total visits, respectively. 'Total visitors' represents the unique individuals visiting the website, whereas 'total visits' account for all the visits made, which may not be unique as a single visitor can make multiple visits. By analyzing these charts, we can directly assess the website's footfall and understand how visitors behave during each action. It's worth noting that in the above funnel charts, the last three actions exhibit variations in their numbers.

###   Checkout details

Let's delve into how smooth our page transactions are by analyzing the actions on the checkout page.

In [6]:
# To get Check out page actions we need to fiter data using action type code 5 and 6 
Query_Checkout_exit_page = """
SELECT PagePath,TotalVisits,TotalExits
FROM(
SELECT
hits.page.pagePath AS PagePath,
COUNT(DISTINCT visitId) AS TotalVisits,
COUNT(hits.isExit) AS TotalExits
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) AS hits
  WHERE 
    hits.eCommerceAction.action_type = '5'
    OR hits.eCommerceAction.action_type = '6'
GROUP BY PagePath 
)
ORDER BY TotalVisits DESC
"""
Query_Checkout_exit_page_df = client.query(Query_Checkout_exit_page).result().to_dataframe()
Query_Checkout_exit_page_df

,PagePath,TotalVisits,TotalExits
0,/yourinfo.html,20896,3546
1,/payment.html,16882,2802
2,/revieworder.html,11674,567
3,/ordercompleted.html,11548,8898
4,/google+redesign/drinkware/mugs+and+cups,1,1


In [7]:
#initialize the plotly figure
fig=go.Figure()
#removing last row from filter view
Query_Checkout_exit_page_df.drop(Query_Checkout_exit_page_df.index[-1], inplace=True)
#Adding the funnel chart traces 
fig.add_trace(go.Funnel(
                        y=Query_Checkout_exit_page_df['PagePath'],
                        x=Query_Checkout_exit_page_df['TotalVisits'],
                        textinfo= "value+percent previous"))
#Update the layout of the chart
fig.update_layout(title_text = 'Check out page view Funnel - % of previous step')
fig.show()

In this funnel, the metrics vary slightly in comparison to the visitor and visit funnels. Here, our focus is on understanding customer actions after completion.
*   On the first page, there were 20,000 visits, out of which Approximately 16,800 visitors submitted their information.
*   Among those who submitted their information, 11,600(aprox.) completed payments.
*   And from those who completed payments, 11,500(aprox.) went on to review and complete their orders."

### Average Time & Pageview

Let's focus on two important metrics: average time spent and pageviews. These metrics are directly related to the interactivity of the website and provide insights into how customers are engaging with it.

In [14]:
 # Average pageviews and time for non transactional sessions, excluding bounse sessions
    Query="""
SELECT 
(SUM(total_pagesviews_per_visit)/COUNT(visit)) AS pageviews_per_visit,
AVG(total_time_per_visit) AS avg_time_per_visit
FROM
(SELECT 
 visitId AS visit,
 AVG(totals.timeOnSite) AS total_time_per_visit,
 SUM(totals.pageviews) AS total_pagesviews_per_visit
 FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE 
   _TABLE_SUFFIX BETWEEN '20170601' AND '20170630'
AND totals.transactions IS NULL
AND totals.bounces IS NULL
GROUP BY visit
)
 """
    
Query_df = client.query(Query).result().to_dataframe()
Query_df

,pageviews_per_visit,avg_time_per_visit
0,5.842755,250.207079


In [17]:
#Average pageviews and time for transactional sessions and bouns sessions excluded 
query='''
        
            SELECT
                AVG(totals.pageviews) AS avg_pageviews_per_visit,
                AVG(totals.timeOnSite) AS avg_time_per_visit,
            FROM`bigquery-public-data.google_analytics_sample.ga_sessions_*`
            WHERE
                _TABLE_SUFFIX BETWEEN '20170601' AND '20170630'
                AND totals.transactions >=1
                AND totals.bounces IS NULL
        '''
query_df = client.query(query).result().to_dataframe()


query_df

,avg_pageviews_per_visit,avg_time_per_visit
0,23.885835,980.30444


Comparing the average time and pageviews between transactional and nontransactional sessions, we observe that transactional sessions tend to have higher average pageviews and more time spent on the website.

# Exit pages

Let's analyze the exit rate for each page.

In [5]:
Query_exit_pages = '''
SELECT Page_path,total_visits,Total_exits,
(Total_exits/total_visits)*100 AS Exit_rate
FROM(
SELECT
hits.page.pagePath AS Page_path,
COUNT(hits.page.pagePath) AS total_visits,
COUNT(hits.isExit) AS Total_exits
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits)AS hits
 WHERE
     _TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
     AND totals.bounces IS NULL
     AND totals.transactions IS NULL
     AND hits.type='PAGE'
 GROUP BY Page_path
)
ORDER BY total_visits DESC
'''

Query_exit_pages_df =client.query(Query_exit_pages).result().to_dataframe()
Query_exit_pages_df.head(10)

,Page_path,total_visits,Total_exits,Exit_rate
0,/home,44617,7294,16.348029
1,/google+redesign/shop+by+brand/youtube,11299,1798,15.912913
2,/basket.html,10919,1566,14.341973
3,/google+redesign/apparel/mens/mens+t+shirts,6685,1280,19.147345
4,/signin.html,6427,837,13.023183
5,/asearch.html,5081,1783,35.091517
6,/store.html,4326,625,14.447527
7,/google+redesign/apparel,3913,656,16.764631
8,/google+redesign/bags,3905,539,13.802817
9,/google+redesign/electronics,3873,745,19.235735


Based on the previous query, we can observe that the home page and YouTube page have higher exit rates, at 16% and 15.9%, respectively. There could be various reasons for this, and one possible explanation could be that ads are reaching untargeted customers.

###  Funnel by Medium

Let's analyze visitor behavior based on different mediums.

In [2]:
by_medium_query='''
SELECT 
 hits.eCommerceAction.action_type AS actions,
trafficSource.medium AS medium,
COUNT(DISTINCT visitId) AS n_visits
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) AS hits
WHERE 
hits.eCommerceAction.action_type ='1'
OR hits.eCommerceAction.action_type ='2'
OR hits.eCommerceAction.action_type ='3'
OR hits.eCommerceAction.action_type ='5'
OR hits.eCommerceAction.action_type ='6'
GROUP BY actions,medium
ORDER BY n_visits DESC
'''
by_medium_query_df=client.query(by_medium_query).result().to_dataframe()
by_medium_query_df.head()

,actions,medium,n_visits
0,1,(none),71095
1,2,(none),70967
2,1,organic,38321
3,2,organic,38269
4,3,(none),30607


In [3]:
action_dict = {'1':'Click product lists', 
              '2':'View product details', 
              '3':'Add to cart', 
              '4':'Remove from cart', 
              '5':'Check out', 
              '6':'Purchase',
              '7':'Refund',
              '0':'Unknown'}
by_medium_query_df['actions']=by_medium_query_df['actions'].map(action_dict)
by_medium_query_df.head(20)

,actions,medium,n_visits
0,Click product lists,(none),71095
1,View product details,(none),70967
2,Click product lists,organic,38321
3,View product details,organic,38269
4,Add to cart,(none),30607
5,Check out,(none),15760
6,Add to cart,organic,13873
7,Purchase,(none),8739
8,Click product lists,referral,8572
9,View product details,referral,8453


In [35]:
bymedium_df =by_medium_query_df
fig = go.Figure()

fig.add_trace(go.Funnel(
    name='Organic',
    y = bymedium_df[bymedium_df['medium']=='organic']['actions'],
    x = bymedium_df[bymedium_df['medium']=='organic']['n_visits'],
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name='Referral',
    orientation='h',
    y = bymedium_df[bymedium_df['medium']=='referral']['actions'],
    x = bymedium_df[bymedium_df['medium']=='referral']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name='Affiliate',
    orientation='h',
    y = bymedium_df[bymedium_df['medium']=='affiliate']['actions'],
    x = bymedium_df[bymedium_df['medium']=='affiliate']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name='CPM',
    orientation='h',
    y = bymedium_df[bymedium_df['medium']=='cpm']['actions'],
    x = bymedium_df[bymedium_df['medium']=='cpm']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name='CPC',
    orientation='h',
    y = bymedium_df[bymedium_df['medium']=='cpc']['actions'],
    x = bymedium_df[bymedium_df['medium']=='cpc']['n_visits'],
    textposition = 'outside',
    textinfo = "value+percent previous"))

fig.update_layout(title_text = 'Visits Funnel by Medium - % of previous step')
fig.show()

By examining the conversion rates of visits through various mediums, it becomes evident that paid mediums such as CPC and CPM exhibit the highest conversion rates, exceeding 60%

For a clearer understanding, we can examine the last three actions in this funnel view.

In [36]:
bymedium_df.drop(bymedium_df[bymedium_df['actions'].isin(['Click product lists', 'View product details'])].index, inplace=True )
bymedium_df.head()

,actions,medium,n_visits
4,Add to cart,(none),30607
5,Check out,(none),15760
6,Add to cart,organic,13873
7,Purchase,(none),8739
10,Check out,organic,4927


In [37]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name='Organic',
    y = bymedium_df[bymedium_df['medium']=='organic']['actions'],
    x = bymedium_df[bymedium_df['medium']=='organic']['n_visits'],
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name='Referral',
    orientation='h',
    y = bymedium_df[bymedium_df['medium']=='referral']['actions'],
    x = bymedium_df[bymedium_df['medium']=='referral']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name='Affiliate',
    orientation='h',
    y = bymedium_df[bymedium_df['medium']=='affiliate']['actions'],
    x = bymedium_df[bymedium_df['medium']=='affiliate']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name='CPM',
    orientation='h',
    y = bymedium_df[bymedium_df['medium']=='cpm']['actions'],
    x = bymedium_df[bymedium_df['medium']=='cpm']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name='CPC',
    orientation='h',
    y = bymedium_df[bymedium_df['medium']=='cpc']['actions'],
    x = bymedium_df[bymedium_df['medium']=='cpc']['n_visits'],
    textposition = 'outside',
    textinfo = "value+percent previous"))

fig.update_layout(title_text = 'Visits Funnel by Medium - Last 3 steps - % of previous step')
fig.show()

### Funnel by Source

Let's examine the visits by their source.

In [49]:
bysource_query="""
SELECT 
 hits.eCommerceAction.action_type AS actions,
trafficSource.source AS source,
COUNT(DISTINCT visitId) AS n_visits
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) AS hits
WHERE 
hits.eCommerceAction.action_type ='1'
OR hits.eCommerceAction.action_type ='2'
OR hits.eCommerceAction.action_type ='3'
OR hits.eCommerceAction.action_type ='5'
OR hits.eCommerceAction.action_type ='6'
GROUP BY actions,Source
ORDER BY n_visits DESC
"""
bysource_df=client.query(bysource_query).result().to_dataframe()
bysource_df.head(10)

,actions,source,n_visits
0,1,(direct),71095
1,2,(direct),70967
2,1,google,40689
3,2,google,40637
4,3,(direct),30607
5,5,(direct),15760
6,3,google,14753
7,6,(direct),8739
8,5,google,5278
9,1,youtube.com,3561


In [50]:
action_dict = {'1':'Click product lists', 
              '2':'View product details', 
              '3':'Add to cart', 
              '4':'Remove from cart', 
              '5':'Check out', 
              '6':'Purchase',
              '7':'Refund',
              '0':'Unknown'}
bysource_df['actions']= bysource_df['actions'].map(action_dict)
bysource_df.head()

,actions,source,n_visits
0,Click product lists,(direct),71095
1,View product details,(direct),70967
2,Click product lists,google,40689
3,View product details,google,40637
4,Add to cart,(direct),30607


In [56]:
fig = go.Figure()

#fig.add_trace(go.Funnel(
 #   name='Google',
  #  y = bysource_df[bysource_df['source']=='google']['actions'],
   # x = bysource_df[bysource_df['source']=='google']['n_visits'],
    #textinfo = "value+percent previous"))

#For Youtube
fig.add_trace(go.Funnel(
    name='Youtube',
     orientation='h',
    y = bysource_df[bysource_df['source']=='youtube.com']['actions'],
    x = bysource_df[bysource_df['source']=='youtube.com']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))

#For Facebook
fig.add_trace(go.Funnel(
    name='Facebook.com',
     orientation='h',
    y = bysource_df[bysource_df['source']=='facebook.com']['actions'],
    x = bysource_df[bysource_df['source']=='facebook.com']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))

#For Yahoo
fig.add_trace(go.Funnel(
    name='Yahoo',
     orientation='h',
    y = bysource_df[bysource_df['source']=='yahoo']['actions'],
    x = bysource_df[bysource_df['source']=='yahoo']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))

Most of the visitors are coming from Google, but there is a slight discrepancy in the data. To enhance the visualization, I've focused on three sources: YouTube, Facebook.com, and Yahoo. Interestingly, Yahoo stands out with a high conversion rate of 10%.

### Paid Ad comparision

It's crucial to evaluate how our investments in terms of both money and time are performing through advertising across different sources. Let's examine this important metric.

In [7]:
# Getting conversion query for CPC and CPM 
paidad_Query='''
SELECT hits.eCommerceAction.action_type AS actions,
                     trafficSource.source AS source,
               COUNT(DISTINCT visitId) AS n_visits
           FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
               UNNEST(hits) as hits
           WHERE
               (trafficSource.medium = 'cpm' 
               OR trafficSource.medium = 'cpc'
               )
               AND(
                  hits.eCommerceAction.action_type = '1'
               OR hits.eCommerceAction.action_type = '2'
               OR hits.eCommerceAction.action_type = '3'
               OR hits.eCommerceAction.action_type = '5'
               OR hits.eCommerceAction.action_type = '6'
               )
           GROUP BY actions, source
           ORDER BY n_visits DESC
           
'''
paidad_Query_df=client.query(paidad_Query).result().to_dataframe()
paidad_Query_df.head()

,actions,source,n_visits
0,1,google,2957
1,2,google,2956
2,1,dfa,1328
3,2,dfa,1327
4,3,google,1108


In [9]:
paidad_Query_df['actions']=paidad_Query_df['actions'].map(action_dic)
paidad_Query_df.head()

,actions,source,n_visits
0,Click product lists,google,2957
1,View product details,google,2956
2,Click product lists,dfa,1328
3,View product details,dfa,1327
4,Add to cart,google,1108


In [10]:
fig = go.Figure()


fig.add_trace(go.Funnel(
    name='Google',
    y = paidad_Query_df[paidad_Query_df['source']=='google']['actions'],
    x = paidad_Query_df[paidad_Query_df['source']=='google']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name='DFA',
    orientation='h',
    y = paidad_Query_df[paidad_Query_df['source']=='dfa']['actions'],
    x = paidad_Query_df[paidad_Query_df['source']=='dfa']['n_visits'],
    textposition = 'inside',
    textinfo = "value+percent previous"))
fig.update_layout(title_text = 'Visits Funnel by CPM/CPC Sources - % of previous step')
fig.show()

In conclusion, the analysis of various metrics and data points has provided valuable insights into our website's performance, visitor behavior, and the effectiveness of our advertising efforts. We've identified strengths, such as the high conversion rate from Yahoo as a traffic source, and areas that require attention, like exit rates on the home page and YouTube. By continuing to monitor and optimize these key metrics, we can make informed decisions to enhance the user experience and maximize the impact of our investments in time and advertising. For more comprehensive information, please refer to the above contents and accompanying analysis.

## Thank you !

Certainly, I understand that you'd like to explore additional papers or sources for more information. Feel free to reach out if you have any specific questions or need assistance with any other topic in the future. I'm here to help.